## Nombres:
- Juan Broto Ortega
- Lidia González Martín

In [2]:
import time
from SPARQLWrapper import SPARQLWrapper, JSON
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, RDFS, OWL
import requests
import sbc_tools as sbc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter


In [30]:
# Configuración del endpoint de Wikidata
WIKIDATA_ENDPOINT = "https://query.wikidata.org/sparql"

class WikidataQuery:
    def __init__(self):
        self.sparql = SPARQLWrapper(WIKIDATA_ENDPOINT)
        self.sparql.setReturnFormat(JSON)
        
        # User-Agent requerido por Wikidata
        self.sparql.addCustomHttpHeader("User-Agent", 
                                       "SBC-Course/1.0 (educational-use)")
    
    def execute_query(self, query):
        """Ejecutar consulta SPARQL con manejo de errores"""
        try:
            self.sparql.setQuery(query)
            results = self.sparql.query().convert()
            return results["results"]["bindings"]
        except Exception as e:
            print(f"Error en la consulta: {e}")
            return []
    
    def results_to_dataframe(self, results):
        """Convertir resultados SPARQL a DataFrame de pandas"""
        if not results:
            return pd.DataFrame()
        
        # Extraer nombres de columnas
        columns = results[0].keys()
        
        # Convertir a diccionario
        data = []
        for result in results:
            row = {}
            for col in columns:
                if col in result:
                    row[col] = result[col]["value"]
                else:
                    row[col] = None
            data.append(row)
        
        return pd.DataFrame(data)

# Instanciar la clase
wd = WikidataQuery()

In [ ]:

URL_PROPIA = "http://librosxxi.org/book/"
ONTO = Namespace("http://librosxxi.org/book-ontology/") 
UMBRAL_JACCARD = 0.3
class BookGenreOntology:
    """
    Clase para construir una ontología RDF de géneros literarios narrativos desde Wikidata.
    """
    
    def __init__(self):
        self.sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
        self.sparql.setReturnFormat(JSON)
        self.graph = Graph()
        
        # Namespaces principales
        self.WD = Namespace("http://www.wikidata.org/entity/")
        self.WDT = Namespace("http://www.wikidata.org/prop/direct/")
        self.BOOK = Namespace("http://librosxxi.org/book-genre/")

        # Vocabs externos
        self.DC = Namespace("http://purl.org/dc/elements/1.1/")
        self.BIBO = Namespace("http://purl.org/ontology/bibo/")
        self.SCHEMA = Namespace("http://schema.org/")
        self.FOAF = Namespace("http://xmlns.com/foaf/0.1/")
        self.ONTO = ONTO

        # Bind al grafo
        self.graph.bind("wd", self.WD)
        self.graph.bind("wdt", self.WDT)
        self.graph.bind("book", self.BOOK)

        self.graph.bind("dc", self.DC)
        self.graph.bind("bibo", self.BIBO)
        self.graph.bind("schema", self.SCHEMA)
        self.graph.bind("foaf", self.FOAF)
        self.graph.bind("onto", self.ONTO)
        
        self.graph.bind("owl", OWL)
        self.graph.bind("rdfs", RDFS)


    def execute_query(self, query):
        """Ejecuta una consulta SPARQL en Wikidata"""
        time.sleep(7)  # Respetar límites
        try:
            self.sparql.setQuery(query)
            results = self.sparql.query().convert()
            return results["results"]["bindings"]
        except Exception as e:
            print(f"Error ejecutando consulta: {e}")
            return []
    

    def build_genre_taxonomy_from_wikidata(self, max_depth: int = None) -> Graph:
        """
        Consulta Wikidata y construye una ontología RDF con la jerarquía de géneros narrativos.
        """
        
        if max_depth is None:
            depth_path = "wdt:P279*"
        else:
            depth_path = "wdt:P279"
            for i in range(1, max_depth):
                depth_path += "/wdt:P279?"

        query = f"""
        SELECT DISTINCT ?genre ?genreLabel ?parent ?parentLabel WHERE {{
          
          ?genre {depth_path} wd:Q1318295 .
          
          ?genre wdt:P279 ?parent .
          ?parent wdt:P279* wd:Q1318295 .
          
          ?genre rdfs:label ?genreLabel .
          FILTER (LANG(?genreLabel) = 'en')
          
          ?parent rdfs:label ?parentLabel .
          FILTER (LANG(?parentLabel) = 'en')
        }}
        ORDER BY ?genreLabel
        """

        bindings = self.execute_query(query)

        if bindings:
            self._build_rdf_graph(bindings)
        
        return self.graph
    

    def _build_rdf_graph(self, bindings: list) -> None:
        print(f"\nProcesando {len(bindings)} resultados...")
        
        # Añadir definición de la clase raíz
        self.graph.add((self.ONTO["LibrosXXI"], RDF.type, OWL.Class))
        self.graph.add((self.ONTO["LibrosXXI"], RDFS.label, Literal("Libros siglo XXI")))
        self.graph.add((self.ONTO["LibrosXXI"], RDFS.subClassOf, self.WD["Q571"])) #Hereda de libro de wikidata 
        self.graph.add((self.ONTO["LibrosXXI"], OWL.equivalentClass, self.DC.BibliographicResource)) ##Equivalente a recurso bibliográfico de Dublin Core
        self.graph.add((self.ONTO["LibrosXXI"], OWL.equivalentClass, self.SCHEMA.Book)) 
        self.graph.add((self.ONTO["LibrosXXI"], OWL.equivalentClass, self.BIBO.Book))
        # Agregamos las propiedades y entidades que correponden a los libros 
        # ------------------ año ------------------
        self.graph.add((self.ONTO.año, RDF.type, OWL.DatatypeProperty)) #Incluimos el año como una propiedad
        self.graph.add((self.ONTO.año, RDFS.label, Literal("Año de publicación")))
        self.graph.add((self.ONTO.año, OWL.equivalentProperty, self.DC.date)) ## Equivalente a fecha de dublin core
        self.graph.add((self.ONTO.año, OWL.equivalentProperty, self.BIBO.created)) ##Equivalente a creado de bibo
        self.graph.add((self.ONTO.año, OWL.equivalentProperty, self.SCHEMA.datePublished)) ##equivalente a la fecha de publicación de schema.org

        # ------------------ ISBN ------------------
        self.graph.add((self.ONTO.isbn, RDF.type, OWL.DatatypeProperty))
        self.graph.add((self.ONTO.isbn, RDFS.label, Literal("ISBN")))
        self.graph.add((self.ONTO.isbn, OWL.equivalentProperty, self.SCHEMA.isbn)) ##Equivalente a isbn de schema.org
        self.graph.add((self.ONTO.isbn, OWL.equivalentProperty, self.BIBO.isbn))  ##equivalente a isbn de bibo

        # ------------------ Género ------------------
        self.graph.add((self.ONTO.Genero, RDF.type, OWL.Class))  ##Creamos una clase para género
        self.graph.add((self.ONTO.Genero, RDFS.label, Literal('Genero')))  ##Damos un Label al género
        self.graph.add((self.ONTO.Genero, RDFS.subClassOf, self.WDT['Q1792379'])) ##Hereda de la clase de wikidata de genero artístico
        self.graph.add((self.ONTO.Genero, OWL.equivalentClass, self.SCHEMA.Genre)) #Equivalente a genero de schema
        self.graph.add((self.ONTO.Genero, OWL.equivalentClass, self.BIBO.DocumentPart)) ##Equivalente a document part de bibo

        self.graph.add((self.ONTO.tieneGenero, RDF.type, OWL.ObjectProperty)) 
        self.graph.add((self.ONTO.tieneGenero, RDFS.domain, self.ONTO["LibrosXXI"])) #Dentro del dominio de libros
        self.graph.add((self.ONTO.tieneGenero, RDFS.range, self.ONTO.Genero)) ## En el rango de genero
        self.graph.add((self.ONTO.tieneGenero, OWL.equivalentProperty, self.SCHEMA.genre)) #equivalente a la propiedad genero
        self.graph.add((self.ONTO.tieneGenero, OWL.equivalentProperty, self.DC.subject)) ## Equivalente a la propiedad tema de dc

        # ------------------ Autor ------------------
        self.graph.add((self.ONTO.Autor, RDF.type, OWL.Class))  ##Creamos una clase autor
        self.graph.add((self.ONTO.Autor, RDFS.label, Literal('Autor')))  ##Damos un Label al autor
        self.graph.add((self.ONTO.Autor, RDFS.subClassOf, self.WDT['Q5'])) ##Hereda de la clase de wikidata de humano
        self.graph.add((self.ONTO.Autor, OWL.equivalentClass, self.BIBO.Person))
        self.graph.add((self.ONTO.Autor, OWL.equivalentClass, self.SCHEMA.Person))
        self.graph.add((self.ONTO.Autor, OWL.equivalentClass, self.FOAF.Person))

        self.graph.add((self.ONTO.tieneAutor, RDF.type, OWL.ObjectProperty))
        self.graph.add((self.ONTO.tieneAutor, RDFS.domain, self.ONTO["LibrosXXI"]))
        self.graph.add((self.ONTO.tieneAutor, RDFS.range, self.ONTO.Autor))
        self.graph.add((self.ONTO.tieneAutor, OWL.equivalentProperty,self.SCHEMA.author))
        self.graph.add((self.ONTO.tieneAutor, OWL.equivalentProperty,self.DC.creator))
        self.graph.add((self.ONTO.tieneAutor, OWL.equivalentProperty,self.BIBO.authorList))

        # ------------------ Editorial ------------------
        self.graph.add((self.ONTO.Editorial, RDF.type, OWL.Class))  # Clase para Editorial
        self.graph.add((self.ONTO.Editorial, RDFS.label, Literal("Editorial")))
        self.graph.add((self.ONTO.Editorial, OWL.equivalentClass, self.SCHEMA.Publisher))  # Equivalente a publisher de schema
        self.graph.add((self.ONTO.Editorial, OWL.equivalentClass, self.BIBO.Publisher))    # Equivalente a publisher de bibo

        self.graph.add((self.ONTO.tieneEditorial, RDF.type, OWL.ObjectProperty))
        self.graph.add((self.ONTO.tieneEditorial, RDFS.domain, self.ONTO["LibrosXXI"]))
        self.graph.add((self.ONTO.tieneEditorial, RDFS.range, self.ONTO.Editorial))
        self.graph.add((self.ONTO.tieneEditorial, OWL.equivalentProperty, self.SCHEMA.publisher))
        self.graph.add((self.ONTO.tieneEditorial, OWL.equivalentProperty, self.DC.publisher))
        self.graph.add((self.ONTO.tieneEditorial, OWL.equivalentProperty, self.BIBO.publisher))


        # ------------------ Tiene formato ebook ------------------
        self.graph.add((self.ONTO.epubAccesibility, RDF.type, OWL.DatatypeProperty))  # Propiedad tipo literal (boolean o string)
        self.graph.add((self.ONTO.epubAccesibility, RDFS.label, Literal("Tiene formato ebook")))
        self.graph.add((self.ONTO.epubAccesibility, OWL.equivalentProperty, self.SCHEMA.bookFormat))


        # ------------------ Descripcion ------------------
        self.graph.add((self.ONTO.descripcion, RDF.type, OWL.DatatypeProperty))  # Propiedad de texto
        self.graph.add((self.ONTO.descripcion, RDFS.label, Literal("Descripción")))
        self.graph.add((self.ONTO.descripcion, OWL.equivalentProperty, self.SCHEMA.description))
        self.graph.add((self.ONTO.descripcion, OWL.equivalentProperty, self.DC.description))


        # ------------------ Maturity rating ------------------
        self.graph.add((self.ONTO.maturityRating, RDF.type, OWL.DatatypeProperty))  # Propiedad tipo literal
        self.graph.add((self.ONTO.maturityRating, RDFS.label, Literal("Maturity Rating")))
        self.graph.add((self.ONTO.maturityRating, OWL.equivalentProperty, self.SCHEMA.contentRating))


        # ------------------ Usuarios ------------------
        self.graph.add((self.ONTO.Usuario, RDF.type, OWL.Class))  # Clase para Usuario
        self.graph.add((self.ONTO.Usuario, RDFS.label, Literal("Usuario")))
        self.graph.add((self.ONTO.Usuario, OWL.equivalentClass, self.SCHEMA.Person))  # Equivalente a Person de schema

        self.graph.add((self.ONTO.edad, RDF.type, OWL.ObjectProperty)) 
        self.graph.add((self.ONTO.edad, RDFS.label, Literal("Edad")))
        self.graph.add((self.ONTO.edad, OWL.equivalentProperty, self.SCHEMA.age))

        self.graph.add((self.ONTO.leGusta, RDF.type, OWL.ObjectProperty)) # Propiedad para relacionar usuario y libro
        self.graph.add((self.ONTO.leGusta, RDFS.domain, self.ONTO.Usuario)) # Dentro del dominio de usuario
        self.graph.add((self.ONTO.leGusta, RDFS.range, self.ONTO["LibrosXXI"])) # En el rango de libros



        # Conjuntos para tracking
        self.genres_added = {}
        
        # Procesar cada resultado
        for i, binding in enumerate(bindings):
            # Extraer información del género
            genre_uri = URIRef(binding['genre']['value'])
            genre_label = binding['genreLabel']['value']
            # Extraer información del padre
            parent_uri = URIRef(binding['parent']['value'])
            parent_label = binding.get('parentLabel', {}).get('value', '')
            
            # Añadir el género como una clase OWL
            if genre_uri not in self.genres_added:

                self.graph.add((genre_uri, RDF.type, self.ONTO.Genero))

                self.graph.add((genre_uri, RDFS.label, Literal(genre_label, lang="en")))
                self.genres_added[genre_uri]=genre_label
            
            # Añadir el padre como una clase OWL
            if parent_uri not in self.genres_added:
                
                self.graph.add((parent_uri, RDF.type, self.ONTO.Genero))

                if parent_label:
                    self.graph.add((parent_uri, RDFS.label, Literal(parent_label, lang="en")))
                self.genres_added[parent_uri]=parent_label
        
            self.graph.add((genre_uri, RDFS.subClassOf, parent_uri))
            
        print(f"Procesamiento completado")

    #Funcion de búsqueda de libros en Google Books
    def get_google_instances(self, genre_label: str, n: int = 5) -> list:
        
        url = f"https://www.googleapis.com/books/v1/volumes?q=subject:{genre_label.replace(' ', '+')}&maxResults={n}"

        resp = requests.get(url)
        data = resp.json()
        sol= []
        try:
            if 'items' in data:
                books = data['items']
                for book in books:
                    res= {}
                    if 'volumeInfo' in book:

                        if 'title' in book['volumeInfo']:
                            res['titulo']=book['volumeInfo']['title']

                        if 'authors' in book['volumeInfo']:
                            autores = []
                            for author in book['volumeInfo']['authors']:
                                autores.append((URIRef(f"{URL_PROPIA}_author={author.replace(' ', '_')}"), author))
                            res['autores']=autores
                        
                        if 'publisher' in book['volumeInfo']:
                            editorial =book['volumeInfo']['publisher']
                            res['editorial'] = (URIRef(f"{URL_PROPIA}_editorial={editorial.replace(' ', '_')}"), editorial)
                        
                        if 'publishedDate' in book['volumeInfo']:
                            res['año']=book['volumeInfo']['publishedDate']

                        if 'description' in book['volumeInfo']:
                            res['description']=book['volumeInfo']['description']

                        if 'industryIdentifiers' in book['volumeInfo']:
                            identifier = book['volumeInfo']['industryIdentifiers'][0] ##Me quedo con el primer ISBN que haya 
                            res['isbn']=identifier['identifier']

                        if 'maturityRating' in book['volumeInfo']:
                            res['maturityRating']=book['volumeInfo']['maturityRating']  
                    if 'epub' in book['accessInfo']:
                        res['epubAccessibility']=book['accessInfo']['epub']['isAvailable']

                    sol.append(res)

                    
        except Exception as e:
            return None
                
        #Devolvemos un lista de diccionarios con la información de los libros
        return sol

    #Función de búsqueda de libros en Wikidata
    def get_wikidata_book_instances(self, genre_uri: str, n: int = 5) -> list:
        genero = genre_uri.split('/')[-1] ##Buscamos por la Q del género
        rm_query = f"""
        PREFIX wd: <http://www.wikidata.org/entity/>
        PREFIX wdt: <http://www.wikidata.org/prop/direct/>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX schema: <http://schema.org/>

        SELECT DISTINCT 
          ?libros 
          ?titulo 
          ?año 
          ?editorial 
          ?es_ebook 
          ?descripcion 
          ?clasificacion_edad 
          ?isbn
          (GROUP_CONCAT(DISTINCT ?autorNombre; separator=";") AS ?autores)
        WHERE {{
            ?libros wdt:P31 wd:Q7725634 ;
                    wdt:P136 wd:{genero} ; # Parámetro de género
                    rdfs:label ?titulo .
            FILTER (lang(?titulo) = "en")

            OPTIONAL {{ 
                ?libros schema:description ?descripcion .
                FILTER (lang(?descripcion) = "en")
            }}

            OPTIONAL {{ 
                ?libros wdt:P50 ?autorEntity .
                ?autorEntity rdfs:label ?autorNombre .
                FILTER (lang(?autorNombre) = "en") 
            }}

            OPTIONAL {{
                ?libros wdt:P123 ?editorialEntity .
                ?editorialEntity rdfs:label ?editorial .
                FILTER (lang(?editorial) = "en")
            }}

            BIND(IF(EXISTS {{ ?libros wdt:P437 wd:Q3331189 }}, "Yes", "No") AS ?es_ebook)

            OPTIONAL {{
                ?libros wdt:P2360 ?targetAudience .
                ?targetAudience rdfs:label ?clasificacion_edad .
                FILTER (lang(?clasificacion_edad) = "en")
            }}

            OPTIONAL {{ 
                ?libros wdt:P577 ?fecha .
                BIND(YEAR(?fecha) AS ?año) 
            }}

            OPTIONAL {{ ?libros wdt:P212 ?isbn . }}
        }}
        GROUP BY ?libros ?titulo ?año ?editorial ?es_ebook ?descripcion ?clasificacion_edad ?isbn
        ORDER BY DESC(?año)
        LIMIT {n}
        """

        wd = WikidataQuery()
        rm_results = wd.execute_query(rm_query)

        libro = {}
        sol = []
        for entity in rm_results:
                if 'titulo' in entity:
                    libro['titulo']=entity["titulo"]["value"]  
                if 'autores' in entity:
                    autores = entity["autores"]["value"].split(";")
                    autores_list = []
                    for author in autores:
                        autores_list.append((URIRef(f"{URL_PROPIA}_author={author.replace(' ', '_').replace(':', '').replace(',', '').replace('(', '').replace(')', '').replace('.', '').replace('"', '').lower()}"), author))
                    libro['autores']=autores_list   

                if 'editorial' in entity:                     
                    editorial =entity["editorial"]["value"]
                    libro['editorial'] = (URIRef(f"{URL_PROPIA}_editorial={editorial.replace(' ', '_').replace(':', '').replace(',', '').replace('(', '').replace(')', '').replace('.', '').replace('"', '').lower()}"), editorial)
                
                if 'año' in entity:
                    libro['año']=entity["año"]["value"]

                if 'descripcion' in entity:
                    libro['description']=entity["descripcion"]["value"]

                if 'isbn' in entity:
                    libro['isbn']=entity["isbn"]["value"]
                if 'clasificacion_edad' in entity:
                    libro['maturityRating']=entity["clasificacion_edad"]["value"]  
                if 'libro' in entity:
                    libro['epubAccessibility']=entity["es_ebook"]["value"]
                sol.append(libro)

        return sol #Retornamos la lista de diccionarios con la información de los libros

    #Función de búsqueda de libros en OpenLibrary
    def get_openLibrary_book_instances(self,genero: str, n: int = 5) -> list:
        # 1. Búsqueda por género
        search_url = f"https://openlibrary.org/subjects/{genero.lower().replace(' ', '_')}.json?limit={n}"
        
        try:
            response = requests.get(search_url)
            data = response.json()
            obras = data.get('works', [])
        except Exception as e:
            return []

        resultados = []

        for obra in obras:
            libro = {}
            
            if obra.get('title'):
                libro["titulo"] = obra.get('title')
            
            autores_list = []
            for author in libro.get('autores', []):
                autores_list.append((URIRef(f"{URL_PROPIA}_author={author.replace(' ', '_').replace(':', '').replace(',', '').replace('(', '').replace(')', '').replace('.', '').replace('"', '').lower()}"), author))
            libro['autores']=autores_list
            
            work_key = obra.get('key')
            edition_key = obra.get('cover_edition_key') or (obra.get('editions')[0].get('key') if obra.get('editions') else None)

            # 2. Detalles de la Obra (Descripción y Edad)
            detalles_obra = requests.get(f"https://openlibrary.org{work_key}.json").json()
            
            # 3. Detalles de la Edición (Editorial, ISBN, Fecha)
            detalles_edicion = {}
            if edition_key:
                path = edition_key if edition_key.startswith('/') else f"/books/{edition_key}"
                detalles_edicion = requests.get(f"https://openlibrary.org{path}.json").json()

            # Fecha de publicación / Año
            anio = detalles_edicion.get('publish_date') or obra.get('first_publish_year')
            if anio:
                libro["año"] = anio

            # Editorial
            pubs = detalles_edicion.get('publishers')
            if pubs and pubs[0]:
                libro["editorial"] = ((URIRef(f"{URL_PROPIA}_editorial={pubs[0].replace(' ', '_').replace(':', '').replace(',', '').replace('(', '').replace(')', '').replace('.', '').replace('"', '').lower()}"), pubs[0]))
            # ISBN (Primero que encuentre)
            isbns = detalles_edicion.get('isbn_13') or detalles_edicion.get('isbn_10')
            if isbns and isbns[0]:
                libro["isbn"] = isbns[0]

            # Accesibilidad Ebook
            if obra.get('has_fulltext'):
                libro["epubAccessibility"] = "Disponible (ePub/PDF)"

            # Clasificación (Solo si se detecta un rango específico)
            subjects = [s.lower() for s in obra.get('subject', [])]
            if any(x in subjects for x in ['erotica', 'explicit', 'adult content']):
                libro["clasificacion"] = "+18"
            elif any(x in subjects for x in ['juvenile', 'children', 'infantil']):
                libro["clasificacion"] = "0-12 años"
            elif "young adult" in subjects:
                libro["clasificacion"] = "13-17 años (Juvenil)"

            # Descripción
            desc_data = detalles_obra.get('description')
            if desc_data:
                descripcion = desc_data.get('value') if isinstance(desc_data, dict) else desc_data
                if descripcion:
                    libro["descripcion"] = descripcion[:400] + "..."

            resultados.append(libro)

        return resultados

    #Función de búsqueda de libros en DBpedia
    def get_dbpedia_book_instances(self, wikidata_genre_uri: str, n: int = 5) -> list:
        url = "https://dbpedia.org/sparql"
        wikidata_id = wikidata_genre_uri.split('/')[-1]
        
        query = f"""
        PREFIX dbo: <http://dbpedia.org/ontology/>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX owl: <http://www.w3.org/2002/07/owl#>
        PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

        SELECT DISTINCT ?book ?title ?desc ?year ?publisher ?isbn (GROUP_CONCAT(DISTINCT ?authorName; separator=", ") AS ?authors)
        WHERE {{
            # Relación entre Wikidata y DBpedia
            {{ ?genre owl:sameAs <http://www.wikidata.org/entity/{wikidata_id}> . }}
            UNION
            {{ ?genre owl:sameAs <https://www.wikidata.org/entity/{wikidata_id}> . }}
            
            # Asociación de libro a género
            {{ ?book dbo:genre ?genre . }} UNION {{ ?book dbo:literaryGenre ?genre . }}
            
            ?book a dbo:Book ;
                rdfs:label ?title .
            FILTER(lang(?title) = "en")

            OPTIONAL {{ 
                ?book dbo:author ?author . 
                ?author rdfs:label ?authorName . 
                FILTER(lang(?authorName) = "en") 
            }}
            OPTIONAL {{ ?book dbo:abstract ?desc . FILTER(lang(?desc) = "en") }}
            OPTIONAL {{ ?book dbo:releaseDate ?date . BIND(year(xsd:date(?date)) AS ?year) }}
            OPTIONAL {{ ?book dbo:publisher ?pub . ?pub rdfs:label ?publisher . FILTER(lang(?publisher) = "en") }}
            OPTIONAL {{ ?book dbo:isbn ?isbn . }}
        }}
        GROUP BY ?book ?title ?desc ?year ?publisher ?isbn
        LIMIT {n}
        """

        try:
            response = requests.get(url, params={'query': query, 'format': 'json'}, timeout=15)
            bindings = response.json().get('results', {}).get('bindings', [])
        except:
            return []

        resultados = []
        for b in bindings:
            libro = {}
            
            # Solo añadimos al diccionario si el campo existe en la respuesta de la API
            if 'title' in b:
                libro["titulo"] = b['title']['value']
            
            if 'authors' in b and b['authors']['value'] != "":
                autores = b['authors']['value'].split(", ")
                autores_list = []
                for author in autores:
                    autores_list.append((URIRef(f"{URL_PROPIA}_author={author.replace(' ', '_').replace(':', '').replace(',', '').replace('(', '').replace(')', '').replace('.', '').replace('"', '').lower()}"), author))
                libro['autores']=autores_list   

                
            if 'year' in b:
                libro["año"] = b['year']['value']
                
            if 'publisher' in b:
                libro["editorial"] = ((URIRef(f"{URL_PROPIA}_editorial={b['publisher']['value'].replace(' ', '_').replace(':', '').replace(',', '').replace('(', '').replace(')', '').replace('.', '').replace('"', '').lower()}"), b['publisher']['value']))
                
            if 'isbn' in b:
                libro["isbn"] = b['isbn']['value']
                
            if 'desc' in b:
                abstract = b['desc']['value']
                libro["descripcion"] = abstract[:400] + "..."
                
                # Clasificación basada en el abstract (si existe)
                lower_abs = abstract.lower()
                if any(x in lower_abs for x in ["erotica", "explicit"]):
                    libro["clasificacion"] = "+18"
                elif any(x in lower_abs for x in ["children's", "juvenile"]):
                    libro["clasificacion"] = "0-12 años"
                elif "young adult" in lower_abs:
                    libro["clasificacion"] = "13-17 años"
            
            resultados.append(libro)

        return resultados #Retornamos la lista de diccionarios con la información de los libros

    #Función para añadir las instancias de libros a la ontología
    def add_instances_to_ontology(self, n_per_genre: int = 50) -> None:
        total_books = 0
        for genre_uri, genre_label in self.genres_added.items():
            try:
                ##Función de búsqueda de libros en Google Books API
                book_instances = self.get_google_instances(genre_label, n_per_genre)
                ##Función de búsqueda de libros en Wikidata
                book_instances_wikidata = self.get_wikidata_book_instances(genre_uri, n_per_genre)
                ##Función de búsqueda de libros en DBpedia
                book_instances_dbpedia = self.get_dbpedia_book_instances(genre_uri, n_per_genre)

                if book_instances:
                    total_books += self._add_books_to_graph(book_instances, genre_uri)
                if book_instances_wikidata:
                    total_books += self._add_books_to_graph(book_instances_wikidata, genre_uri)
                if book_instances_dbpedia:
                    total_books += self._add_books_to_graph(book_instances_dbpedia, genre_uri)
                print(total_books)
            except Exception as e:
                pass #En caso de error seguimos con el siguiente género
        print(f"Total libros añadidos: {total_books}")

    #Función interna para añadir libros al grafo RDF
    def _add_books_to_graph(self, rm_results: dict, genre_uri: URIRef) -> int:
    
        count = 0
        for entity in rm_results:
            # URI y label del libro (obligatorio)
            entity_uri =  URIRef(f"{URL_PROPIA}_titulo={entity["titulo"].replace(' ', '_').replace(':', '').replace(',', '').replace('(', '').replace(')', '').replace('.', '').replace('"', '').lower()}")
            if (entity_uri, RDF.type, self.ONTO["LibrosXXI"]) in self.graph:
                continue  # Ya existe no lo metemos otra vez
            entity_label = entity["titulo"]
            
            self.graph.add((entity_uri, RDF.type, self.ONTO["LibrosXXI"])) #Tipo libro siglo XXI

            self.graph.add((entity_uri, RDFS.label, Literal(entity_label))) #Label del libro

            self.graph.add((entity_uri, self.ONTO.tieneGenero, genre_uri)) #Relación con el género

  
            if 'año' in entity: 
                self.graph.add((entity_uri, self.ONTO.año, Literal(entity['año'])))

            if 'isbn' in entity:
                self.graph.add((entity_uri, self.ONTO.isbn, Literal(entity['isbn'])))

            if 'description' in entity:
                self.graph.add((entity_uri, self.ONTO.description, Literal(entity['description'])))

            if 'maturityRating' in entity:
                self.graph.add((entity_uri, self.ONTO.maturityRating, Literal(entity['maturityRating'])))

            if 'epubAccessibility' in entity:
                self.graph.add((entity_uri, self.ONTO.epubAccesibility, Literal(entity['epubAccessibility'])))

            #Meto cada autor como una instancia con su uri y label y lo relaciono con el libro 
            if 'autores' in entity:
                for autor in entity['autores']:
                    self.graph.add((autor[0], RDF.type, self.ONTO["Autor"]))
                    self.graph.add((autor[0], RDFS.label, Literal((autor[1]))))
                    self.graph.add((entity_uri, self.ONTO.tieneAutor, autor[0]))

            #Meto cada editorial como una instancia con su uri y label y lo relaciono con el libro 
            if 'editorial' in entity:
                self.graph.add((entity['editorial'][0], RDF.type, self.ONTO["Editorial"]))
                self.graph.add((entity['editorial'][0], RDFS.label, Literal(entity['editorial'][1])))
                self.graph.add((entity_uri, self.ONTO.tieneEditorial, entity['editorial'][0]))
            
            count += 1

        return count

In [32]:
book_onto = BookGenreOntology()
rdf_graph = book_onto.build_genre_taxonomy_from_wikidata(max_depth=3)


Procesando 1238 resultados...
Procesamiento completado


In [ ]:
book_onto.add_instances_to_ontology(50) #Añadir hasta 50 libros por género

In [36]:
sbc.save(book_onto.graph, "grafo_save.html", format="turtle")

Ontología guardada en: data\grafo_save.html


In [37]:
sbc.save(book_onto.graph, "grafo_save_ttl.ttl", format="turtle")

Ontología guardada en: data\grafo_save_ttl.ttl


In [ ]:
sbc.show_graph(rdf_graph, "grafo_save.html")

out/grafo_recomendador.html


# Similitudes

In [3]:
##Cargamos el grafo
graph = sbc.load("grafo_save.html", format="turtle")

In [4]:
sbc.show_graph(graph, "grafo_save.html")

out/grafo_save.html


## Recomendacion dado un libro

In [5]:
from pyparsing import deque

#Funcion para calcular la distancia entre dos libros. Retorna la distancia y el camino
def find_path_and_distance(graph, book1_uri, book2_uri, max_depth=10):

    if book1_uri == book2_uri:
        return 0, [book1_uri]
    
    adj_graph = {}
    def add_edge(u, v):
        adj_graph.setdefault(u, set()).add(v)
        adj_graph.setdefault(v, set()).add(u)

    for s, p, o in graph.triples((None, ONTO.tieneGenero, None)):
        if isinstance(s, URIRef) and isinstance(o, URIRef): add_edge(s, o)
        
    for s, p, o in graph.triples((None, RDFS.subClassOf, None)):
        if isinstance(s, URIRef) and isinstance(o, URIRef): add_edge(s, o)

    # El BFS ahora guarda el camino: (nodo_actual, distancia, camino_recorrido)
    visited = {book1_uri}
    queue = deque([(book1_uri, 0, [book1_uri])])

    while queue:
        current, distance, path = queue.popleft()
        if distance >= max_depth: continue

        for neighbor in adj_graph.get(current, set()):
            if neighbor == book2_uri:
                return distance + 1, path + [neighbor]
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append((neighbor, distance + 1, path + [neighbor]))
    
    return -1, []

In [8]:
#Codigo de prueba de la función de distancia
book1_uri = URIRef("http://librosxxi.org/book/_titulo=amanecer")
book2_uri = URIRef("http://librosxxi.org/book/_titulo=a_man")
find_path_and_distance(graph, book1_uri, book2_uri, max_depth=10)

(7,
 [rdflib.term.URIRef('http://librosxxi.org/book/_titulo=amanecer'),
  rdflib.term.URIRef('http://www.wikidata.org/entity/Q11820949'),
  rdflib.term.URIRef('http://www.wikidata.org/entity/Q474090'),
  rdflib.term.URIRef('http://www.wikidata.org/entity/Q948970'),
  rdflib.term.URIRef('http://www.wikidata.org/entity/Q8261'),
  rdflib.term.URIRef('http://www.wikidata.org/entity/Q3056541'),
  rdflib.term.URIRef('http://www.wikidata.org/entity/Q4914883'),
  rdflib.term.URIRef('http://librosxxi.org/book/_titulo=a_man')])

In [ ]:
import random
#Función de recomendación de libros con pesos
def recommend_weighted_books(graph, target_book_uri, top_n=5, randomness=0.1):
  
    #pesos 
    WEIGHT_AUTHOR = 0.5    # Mucha importancia si es el mismo autor
    WEIGHT_PUBLISHER = 0.2 # Importancia media si es la misma editorial
    # La similitud de género (distancia) será la base (máximo 1.0)
    
    all_books = set()
    for s, p, o in graph.triples((None, RDF.type, ONTO["LibrosXXI"])):
        if s != target_book_uri:
            all_books.add(s)
            
    # Extraer metadatos del libro objetivo para comparar
    target_authors = set(graph.objects(target_book_uri, ONTO.tieneAutor))
    target_pub = graph.value(target_book_uri, ONTO.tieneEditorial)

    candidates = []
    for book_uri in all_books:
        #Similitud de genero
        dist, path = find_path_and_distance(graph, target_book_uri, book_uri)
        if dist == -1: continue # Si no hay conexión alguna, ignorar
        
        genre_sim = 1 / (dist + 1)
        
        #Bonus por autor comun, tengo en cuenta que puede haber varios autores
        author_bonus = 0
        current_authors = set(graph.objects(book_uri, ONTO.tieneAutor))
        if target_authors.intersection(current_authors):
            author_bonus = WEIGHT_AUTHOR
            
        #Bonus por editorial comun
        pub_bonus = 0
        current_pub = graph.value(book_uri, ONTO.tieneEditorial)
        if target_pub and current_pub and target_pub == current_pub:
            pub_bonus = WEIGHT_PUBLISHER
            
        #agregamos un bonus de aleatoriedad para tener algo de serendipia
        luck = random.uniform(0, randomness)
        final_score = genre_sim + author_bonus + pub_bonus + luck
        
        label = graph.value(book_uri, RDFS.label) or str(book_uri).split('=')[-1]
        candidates.append({
            'label': label,
            'score': round(final_score, 3),
            'reasons': {
                'same_author': author_bonus > 0,
                'same_pub': pub_bonus > 0,
                'genre_dist': dist, 
                'path': path
            }
        })

    #ordenados por score y devolvemos los top 
    candidates.sort(key=lambda x: x['score'], reverse=True)
    return candidates[:top_n]

In [21]:
##Procesa una recomendacion de ejemplo y la devuelve en texto comprensible
def explain_recommendations(graph, target_book_uri, recommendations):
    target_label = graph.value(target_book_uri, RDFS.label) or "Libro seleccionado"
    print(f"\n--- Explicación para: {target_label} ---")
    
    for rec in recommendations:
        print(f"\nRECOMENDACIÓN: {rec['label']} (Score: {rec['score']})")
        
        # Explicar Género y Camino
        path = rec['reasons']['path']
        dist = rec['reasons']['genre_dist']
        
        # Traducir URIs a etiquetas para el camino
        labels_path = []
        for uri in path:
            l = graph.value(uri, RDFS.label)
            labels_path.append(str(l) if l else str(uri).split('/')[-1])
        
        print(f"  • Género: Distancia {dist}. Camino semántico: {' -> '.join(labels_path)}")
        
        # Explicar Bonus
        if rec['reasons']['same_author']:
            print(f"  • Bonus Autor: Coincidencia de autor detectada (+0.5)")
        if rec['reasons']['same_pub']:
            print(f"  • Bonus Editorial: Ambos publicados por la misma editorial (+0.2)")

In [22]:
libro_ejemplo = URIRef("http://librosxxi.org/book/_titulo=amanecer")
recomendacion = recommend_weighted_books(graph, libro_ejemplo, top_n=5)
explain_recommendations(graph, libro_ejemplo, recomendacion)


--- Explicación para: Amanecer ---

RECOMENDACIÓN: Divine Comedy (Score: 0.348)
  • Género: Distancia 3. Camino semántico: Amanecer -> philosophical poem -> narrative poetry -> Divine Comedy

RECOMENDACIÓN: Máj (Score: 0.341)
  • Género: Distancia 3. Camino semántico: Amanecer -> philosophical poem -> narrative poetry -> Máj

RECOMENDACIÓN: Willobie His Avisa (Score: 0.338)
  • Género: Distancia 3. Camino semántico: Amanecer -> philosophical poem -> narrative poetry -> Willobie His Avisa

RECOMENDACIÓN: Childe Harold's Pilgrimage (Score: 0.336)
  • Género: Distancia 3. Camino semántico: Amanecer -> philosophical poem -> narrative poetry -> Childe Harold's Pilgrimage

RECOMENDACIÓN: Poltava (poem) (Score: 0.329)
  • Género: Distancia 3. Camino semántico: Amanecer -> philosophical poem -> narrative poetry -> Poltava (poem)


## Agregamos usuarios

In [23]:
##Función para agregar usuarios al grafo
def agregar_usuarios(graph, usuario):
        usuario_uri = URIRef(f"{URL_PROPIA}_usuario={usuario['nombre']}")
        graph.add((usuario_uri, RDF.type, ONTO.Usuario))
        graph.add((usuario_uri, RDFS.label, Literal(usuario['nombre'])))
        graph.add((usuario_uri, ONTO.edad, Literal(usuario['edad'])))

        for libro_uri in usuario['libros_gustados']:
            graph.add((usuario_uri, ONTO.leGusta, libro_uri))

## Generamos 10 usuarios para hacer pruebas

In [24]:
import random
from rdflib import URIRef

#Listado de libros disponibles
libros_disponibles = [
    "12 Days (book)",
    "20 Hrs. 40 Min.",
    "25 Images of a Man's Passion",
    "365 Read-Aloud Bedtime Bible Stories",
    "44 Scotland Street",
    "62: A Model Kit",
    "69 (novel)",
    "99 Fables",
    "A Bell for Adano (novel)",
    "A boccaperta",
    "A Bridegroom at Fourteen",
    "A Buyer's Market",
    "A Cat Abroad",
    "A Charge to Keep",
    "A Choice of Magic",
    "A Closed Book (novel)",
    "A Confederacy of Dunces",
    "A Daughter's a Daughter",
    "A Demon in My View",
    "A Dog of Flanders",
    "A Face Like Glass",
    "A First-Class Story; or, The Perils of Travelling Alone",
    "A Flight of Pigeons",
    "A Fortunate Life",
    "A Friend of the Family (novel)",
    "A Girl Is a Half-formed Thing",
    "A God Strolling in the Cool of the Evening",
    "A Good Clean Fight",
    "A Hope in the Unseen",
    "A Journey Around My Room",
    "A Journey to Lhasa and Central Tibet",
    "A Journey to the Rivers",
    "A Lady's Life in the Rocky Mountains",
    "A Life of Contrasts",
    "A Lineage of Grace",
    "A Lion's Tale",
    "A London Life",
    "A Man",
    "A Man Was Going Down the Road",
    "A Narrative of the Captivity and Restoration of Mrs. Mary Rowlandson",
    "A Novel about a Good Person",
    "A Passionate Pilgrim",
    "A Peep Behind the Scenes (novel)",
    "A Riddle of Roses",
    "A Rose Beyond the Thames",
    "A Sentimental Journey Through France and Italy",
    "A Separate Reality",
    "A Short History of the Confederate States of America",
    "A Son of the Soil",
    "A Stranger to Myself: The Inhumanity of War: Russia, 1941–1944",
    "A Tale of a Tub",
    "A Thousand Tomorrows",
    "A Tourist in Africa",
    "A Tramp Abroad",
    "A Traveller in Time",
    "A Watcher in the Woods",
    "A White House Diary",
    "A Wizard of Earthsea",
    "A Woman's Burden",
    "A World for Julius (novel)",
    "A World with No Shore",
    "Abel",
    "Abeng (novel)",
    "Absent in the Spring",
    "Accelerando",
    "Acoso textual",
    "Acts of God (novel)",
    "A.D.: New Orleans After the Deluge",
    "Adrista",
    "Adventures of Wim",
    "Aesop's Fables (Pinkney book)",
    "Aetnaeae",
    "After the Plague",
    "Agatha Christie's Secret Notebooks",
    "Aki-wayn-zih",
    "Alaung Mintayagyi Ayedawbon",
    "Alaungpaya Ayedawbon",
    "Albidaro and the Mischievous Dream",
    "Alfred Hitchcock's Anthology – Volume 5",
    "Alice, I Think (novel)",
    "Alice in Sunderland",
    "Aline and Valcour",
    "All Families Are Psychotic",
    "All God's Children Need Traveling Shoes",
    "All Heads Turn When the Hunt Goes by",
    "All in a Lifetime",
    "All Quiet on the Orient Express",
    "All Quiet on the Western Front",
    "All the World's Mornings",
    "Alzheimer's Story",
    "Amanecer",
    "Amazon Adventure",
    "Amelia Rules!",
    "American Born Chinese (graphic novel)",
    "American Menu",
    "Among the Believers",
    "Among the Betrayed",
    "Amuktamalyada",
    "An American Demon",
    "An American Tragedy",
    "An Area of Darkness",
    "An Awfully Big Adventure (novel)",
    "An Ice-Cream War",
    "Anatomy of a Disappearance",
    "Anchu (novel)",
    "Anecdotes of Oyasama",
    "Angelus ad aras",
    "Angus, Thongs and Full-Frontal Snogging",
    "Angéline de Montbrun",
    "Anon Pls.",
    "Anzukko",
    "Apache",
    "Aplec de Rondalles Mallorquines d'en Jordi des Racó",
    "Apollonius of Tyre"
]

#creamos las uris de los libros
libros_uris = {titulo: URIRef(f"{URL_PROPIA}_titulo={titulo.replace(' ', '_').replace(':', '').replace(',', '').replace('(', '').replace(')', '').replace('.', '').lower()}") 
               for titulo in libros_disponibles}

##Seleccionamos un par de libros para que coincidan y los usuarios de prueba tengan similitudes
libros_populares = [
    "A Confederacy of Dunces",
    "All Quiet on the Western Front",
    "A Wizard of Earthsea",
    "Accelerando",
    "An American Tragedy",
    "Aesop's Fables (Pinkney book)",
    "Alice in Sunderland",
    "American Born Chinese (graphic novel)",
    "After the Plague",
    "All Families Are Psychotic"
]

#generamos 10 usuarioos
usuarios = []
nombres = ['juan_perez', 'maria_garcia', 'carlos_lopez', 'ana_martinez', 'luis_rodriguez',
           'laura_hernandez', 'pedro_gonzalez', 'sofia_diaz', 'miguel_sanchez', 'elena_ruiz']

for i in range(10):
    #cada usuario tendrá 10 libros gustados
    #mezcla de libros populares (30-50%) y libros aleatorios
    num_populares = random.randint(3, 5)
    seleccion_populares = random.sample(libros_populares, num_populares)
    
    #completar con libros aleatorios
    libros_restantes = [libro for libro in libros_disponibles if libro not in seleccion_populares]
    seleccion_aleatoria = random.sample(libros_restantes, 10 - num_populares)
    
    libros_usuario = seleccion_populares + seleccion_aleatoria
    
    #crear lista de uris
    libros_uris_usuario = [libros_uris[libro] for libro in libros_usuario]
    
    usuario = {
        'nombre': nombres[i],
        'edad': random.randint(20, 65),
        'libros_gustados': libros_uris_usuario
    }
    
    usuarios.append(usuario)

for usuario in usuarios:
    agregar_usuarios(graph, usuario)


In [25]:
sbc.save(graph, "grafo_con_usuarios.ttl", format="turtle")

Ontología guardada en: data\grafo_con_usuarios.ttl


In [26]:
sbc.save(graph, "grafo_con_usuarios.html", format="turtle")

Ontología guardada en: data\grafo_con_usuarios.html


In [30]:
from rdflib import Namespace

ONTO = Namespace("http://librosxxi.org/book-ontology/")

usuario_x = URIRef("http://librosxxi.org/book/_usuario=juan_perez")

query = f"""
SELECT DISTINCT ?otroUsuario ?libro
WHERE {{
  <{usuario_x}> onto:leGusta ?libro .
  ?otroUsuario onto:leGusta ?libro .
  FILTER (?otroUsuario != <{usuario_x}>)
}}
"""

usuarios_similares = {}

for row in graph.query(query, initNs={"onto": ONTO}):
    otro = row.otroUsuario
    libro = row.libro
    usuarios_similares.setdefault(otro, set()).add(libro)

for u, libros in usuarios_similares.items():
    print(u, "comparte libros:", libros)


http://librosxxi.org/book/_usuario=maria_garcia comparte libros: {rdflib.term.URIRef('http://librosxxi.org/book/_titulo=a_wizard_of_earthsea'), rdflib.term.URIRef('http://librosxxi.org/book/_titulo=american_born_chinese_graphic_novel')}
http://librosxxi.org/book/_usuario=luis_rodriguez comparte libros: {rdflib.term.URIRef('http://librosxxi.org/book/_titulo=all_quiet_on_the_western_front'), rdflib.term.URIRef('http://librosxxi.org/book/_titulo=a_wizard_of_earthsea'), rdflib.term.URIRef('http://librosxxi.org/book/_titulo=365_read-aloud_bedtime_bible_stories')}
http://librosxxi.org/book/_usuario=laura_hernandez comparte libros: {rdflib.term.URIRef('http://librosxxi.org/book/_titulo=a_confederacy_of_dunces'), rdflib.term.URIRef("http://librosxxi.org/book/_titulo=a_buyer's_market"), rdflib.term.URIRef('http://librosxxi.org/book/_titulo=all_quiet_on_the_western_front'), rdflib.term.URIRef('http://librosxxi.org/book/_titulo=a_wizard_of_earthsea'), rdflib.term.URIRef("http://librosxxi.org/book

## Función de Jaccard

In [31]:
#Función para calcular la similitud de Jaccard entre dos usuarios
def jaccard_users(graph, user1_uri, user2_uri):
    """
    Calcula la similitud de Jaccard entre dos usuarios
    """
    libros_u1 = set(graph.objects(user1_uri, ONTO.leGusta))
    libros_u2 = set(graph.objects(user2_uri, ONTO.leGusta))
    
    if not libros_u1 and not libros_u2:
        return 0.0
    
    interseccion = libros_u1.intersection(libros_u2)
    union = libros_u1.union(libros_u2)
    
    return round(len(interseccion) / len(union), 3)


In [32]:
#Umbral de similitud para considerar usuarios similares
UMBRAL_JACCARD = 0.3
usuarios = set(graph.subjects(RDF.type, ONTO.Usuario))
jaccard_results = []

for user in usuarios:
    if user == usuario_x:
        continue  # Omitir el usuario de referencia
    sim = jaccard_users(graph, usuario_x, user)
    if sim > UMBRAL_JACCARD: 
        jaccard_results.append((usuario_x, user, sim))

libros_usuario = {}
libros_originales = set(graph.objects(usuario_x, ONTO.leGusta))

for u1, u2, sim in sorted(jaccard_results, key=lambda x: x[2], reverse=True):
    label_u1 = graph.value(u1, RDFS.label)
    label_u2 = graph.value(u2, RDFS.label)
    libros_usuario2= set(graph.objects(u2, ONTO.leGusta))

    for libro in libros_usuario2:
        if libro not in libros_originales:
            if libro in libros_usuario:
                libros_usuario[libro] += sim
            else:
                libros_usuario[libro] = sim
     
    print(f"{label_u1} ↔ {label_u2} → Jaccard = {sim}")

print(libros_usuario)


juan_perez ↔ laura_hernandez → Jaccard = 0.333
{rdflib.term.URIRef('http://librosxxi.org/book/_titulo=99_fables'): 0.333, rdflib.term.URIRef('http://librosxxi.org/book/_titulo=abeng_novel'): 0.333, rdflib.term.URIRef('http://librosxxi.org/book/_titulo=among_the_believers'): 0.333, rdflib.term.URIRef('http://librosxxi.org/book/_titulo=anon_pls'): 0.333, rdflib.term.URIRef('http://librosxxi.org/book/_titulo=a_london_life'): 0.333}
